In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import os
from openpyxl import load_workbook

In [ ]:
del_dir1='test'
del_dir2='bimodal gene number in different condition'
filelist=['Danish', 'Spanish', 'American','3_countries_combined', 'Human_nose','Human_cheek', 'Human_forehead', 'Pig','Mouse','Marine']
file_path = 'Bimodal Gene numbers(silverman).xlsx'
Bimodal_Gene_numbers = pd.read_excel(del_dir2+'/'+file_path, sheet_name='Sheet1', index_col=0)

N=[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
M=[0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4]
for n in range(len(N)):
    for m in range(len(M)):
        for k in range(len(filelist)):
            del_file=del_dir1+'/'+filelist[k]
            df=pd.read_csv(del_file,index_col=0,delimiter='\t') 
            dT=df.T
            #Finding Biomodal Distribution of genes
            Bimodal_genenames=[]
            for i in range(len(dT.columns)):
                y=dT.iloc[:,i]
                if np.sum(abs(np.log(y[y>0]))>0)>0.3*len(dT): # Limit the number of non-zero samples to be large enough
                    data = np.log(y[y>0]) 
                    #sns.kdeplot(data,bw_adjust=1) 
                    kde = gaussian_kde(data,bw_method='silverman') # Get the kernel density function result('silverman')
                    kde_data = kde.evaluate(np.linspace(min(data), max(data), 1000))  
                    kde_x=np.linspace(min(data), max(data), 1000) 
                    kde_y=kde(kde_x) 
                    dy_dx = np.gradient(kde_y, kde_x) # Differentiate the kernel density function.
                    peakposi_y=[] # y-values of peaks
                    valeyposi_y=[] # y-values of valleys
                    peakposi_x=[] # x-values of peaks
                    valeyposi_x=[] # x-values of valleys
                    for j in range(len(dy_dx)):
                        if dy_dx[j-1]>=0 and dy_dx[j]<=0 :  # Use the derivative equal to 0 as the screening condition for peaks and valleys.
                            peak_y=kde_y[j-1]
                            peak_y=kde_y[j-1]
                            peakposi_y.append(peak_y)
                            peak_x=kde_x[j-1]
                            peakposi_x.append(peak_x)
                        if dy_dx[j-1]<=0 and dy_dx[j]>=0:
                            valey_y=kde_y[j]
                            valeyposi_y.append(valey_y)
                            valey_x=kde_x[j]
                            valeyposi_x.append(valey_x)
                    if len(peakposi_y)>=2: # Screen out cases with two or more peaks
                        Ymax1=max(peakposi_y) # Maximum peak value
                        BB=[]
                        for a in peakposi_y: 
                            BB.append(a)
                        BB.remove(max(BB))# Remove the maximum peak value from the peak value set.
                        Ymax2=max(BB) 
                        Xmax1=peakposi_x[peakposi_y.index(Ymax1)] # x-value corresponding to the maximum peak value
                        Xmax2=peakposi_x[peakposi_y.index(Ymax2)] # x-value corresponding to the second peak value
                        Yvaley=[] # Data set of y-values of valleys between the maximum peak and the second peak
                        for b in valeyposi_x:
                            if Xmax1<b<=Xmax2 or Xmax2<b<=Xmax1:
                                CC=valeyposi_y[valeyposi_x.index(b)]
                                Yvaley.append(CC)

                        # Screening conditions for bimodal distribution
                        if Ymax2>=N[n]*Ymax1 and (Ymax2-min(Yvaley))>=M[m]*Ymax2 : 
                            posi=i
                            Bimodal_genename=dT.columns[i] 
                            Bimodal_genenames.append(Bimodal_genename)
             # Load the workbook
            wb = load_workbook(del_dir2+'/'+file_path)
            # Select the worksheet to operate on
            sheet = wb['Sheet1']  
            # Define the starting row and column numbers to write
            rows = 3
            cols = 4
            sheet.cell(row=rows+k, column=cols+m+n*len(M)).value = len(Bimodal_genenames)
            # Convert the list to a comma-separated string
            Bimodal_genenames_str = ', '.join(Bimodal_genenames)
            sheet.cell(row=rows+k+len(filelist), column=cols+m+n*len(M)).value = Bimodal_genenames_str
            wb.save(del_dir2+'/'+file_path)
            print(f'Written to row:{rows+k}, column:{cols+m+7*n}')